In [2]:
from langchain_community.document_loaders import JSONLoader

# Lade dein FAQ-Dokument
loader = JSONLoader(
    file_path="/home/niels/dev/cora_customer_agent/company_faq.json",
    jq_schema=".[]",
    content_key="content",
)

docs = loader.load()
print(docs[0])

page_content='Question: How can I place an order on TechHive?
Answer: Select your desired products, add them to your cart, and follow the checkout process. You can create a customer account or order as a guest.' metadata={'source': '/home/niels/dev/cora_customer_agent/company_faq.json', 'seq_num': 1}


In [ ]:
# from langchain_openai import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/home/niels/dev/cora_customer_agent/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="company_faq",
    embedding_function=embeddings,
    host="localhost",  
    port=8000  
)

In [9]:
ids = vector_store.add_documents(documents=docs)

In [14]:
results = await vector_store.asimilarity_search(
    "Which payment methods does TechHive accept?", k=1
)

print(results[0].page_content)

Question: Which payment methods does TechHive accept?
Answer: We accept credit cards, PayPal, Klarna, instant bank transfer, Apple Pay, and Google Pay.
